# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import modules.data_analysis_utils as dau
from importlib import reload

# my_computer_fpath = "C:\\Users\\dfber\\OneDrive - Mass General Brigham\\Epidural project\\Data\\"
my_computer_fpath = "C:\\Users\\User\\OneDrive - Mass General Brigham\\Epidural project\\Data\\"

# Load data

In [ ]:
df = pd.read_csv(my_computer_fpath + 'processed_and_imputed_merlin_data.csv') 

## Prepend '#' for better dummies

In [ ]:
reload(dau)

In [ ]:
df = dau.prepend_char_to_most_common(df, '#', cols_to_ignore=['anes_procedure_encounter_id_2273','unique_pt_id'])

In [ ]:
neuraxial_catheter_df = df.copy()

# Delete unknowable columns to prevent data leakage

In [ ]:
data_leakage_columns = ['rom_thru_delivery_hours','placement_to_delivery_hours']
neuraxial_catheter_df = neuraxial_catheter_df.drop(data_leakage_columns, axis=1,errors='ignore')

# Statistical Analysis

## Some individually interesting regressions

In [ ]:
df_corr = neuraxial_catheter_df.dropna(subset=['lor_depth', 'number_of_neuraxial_attempts'])

# Fit the model using the formula
model = smf.ols('number_of_neuraxial_attempts ~ lor_depth', data=df_corr).fit()

# Print the summary of the regression results
print(model.summary())


In [ ]:
# prompt: Do univariate logistic regression separately using number of attempts and loss of resistance depth to predict failure

import statsmodels.api as sm
import statsmodels.formula.api as smf

# Prepare the data for logistic regression with number of attempts as the predictor
df_logreg_attempts = neuraxial_catheter_df.dropna(subset=['number_of_neuraxial_attempts', 'failed_catheter'])
# Fit the logistic regression model
model_attempts = smf.logit('failed_catheter ~ number_of_neuraxial_attempts', data=df_logreg_attempts).fit()

# Print the summary of the model
print(model_attempts.summary())


# Prepare the data for logistic regression with loss of resistance depth as the predictor
df_logreg_lor = neuraxial_catheter_df.dropna(subset=['lor_depth', 'failed_catheter'])
# Fit the logistic regression model
model_lor = smf.logit('failed_catheter ~ lor_depth', data=df_logreg_lor).fit()

# Print the summary of the model
print(model_lor.summary())


In [ ]:
# prompt: Now do multivariate analysis using the same two predictors

# Prepare the data for logistic regression with both predictors
df_logreg_multi = neuraxial_catheter_df.dropna(subset=['number_of_neuraxial_attempts', 'lor_depth', 'failed_catheter'])

# Fit the logistic regression model with both predictors
model_multi = smf.logit('failed_catheter ~ number_of_neuraxial_attempts + lor_depth', data=df_logreg_multi).fit()

# Print the summary of the model
print(model_multi.summary())


In [ ]:
# Prepare the data for logistic regression with prior_failed_catheters_this_enc as the predictor
df_logreg_prior_failed = neuraxial_catheter_df.dropna(subset=['prior_failed_catheters_this_enc', 'failed_catheter'])

# Fit the logistic regression model
model_attempts = smf.logit('failed_catheter ~ prior_failed_catheters_this_enc', data=df_logreg_prior_failed).fit()

# Print the summary of the model
print(model_attempts.summary())

## All univariate regressions

In [ ]:
results_df = dau.all_regressions_each_dummy(neuraxial_catheter_df, 'failed_catheter')
# This returns a DataFrame with columns: [column, param_name, coef, pval].
# Each level of a categorical predictor will appear as a separate row.

In [ ]:
results_df['category_variable'] = results_df['param_name'].apply(dau.parse_param_name)

In [ ]:
results_df[['column','category_variable','coef','pval']]

In [ ]:
results_df.shape

In [ ]:
results_df[results_df['pval'] < 0.05 / results_df.shape[0]].shape

In [ ]:
reload(dau)

In [ ]:
# Create plot: coefficient vs -log10(p-value)
fig, ax = plt.subplots(figsize=(8, 6))

offset = 1e-300  # so we don't take log10(0)
x_vals = results_df[results_df['pval'] < 0.9]['coef']
y_vals = -np.log10(results_df[results_df['pval'] < 0.9]['pval'] + offset)

sc = ax.scatter(x_vals, y_vals, color='blue')

# Annotate each point
for i, row in results_df[results_df['pval'] < 0.9].iterrows():
    ax.text(
        row['coef'],
        -np.log10(row['pval'] + offset),
        dau.remove_nums(str(row['column'] + ('__' + str(row['category_variable'])) if row['category_variable'] != '' else row['column'])),
        fontsize=8,
        ha='left',
        va='bottom'
    )

# Add a reference line for p=0.05
ax.axhline(-np.log10(0.05), color='red', linestyle='--', label='p=0.05')

ax.set_xlabel('Coefficient')
ax.set_ylabel('-log10(p-value)')
ax.set_title(f'Logistic Regressions for Catheter_Failure ~ Each Predictor (All Dummies)')
ax.legend()

plt.tight_layout()
plt.show()


# Scale numerical values

In [ ]:
for col in ['bmi_end_pregnancy_2044', 'baby_weight_2196', 'gestational_age_weeks', 'maternal_age_years']:
    neuraxial_catheter_df[col] = neuraxial_catheter_df[col] - neuraxial_catheter_df[col].median()

In [ ]:
neuraxial_catheter_df['gestational_age_weeks'].describe()

# Logistic Regression Model

### Random data for model comparison

In [ ]:
test_dataset = neuraxial_catheter_df.copy()
failure_rate = test_dataset['failed_catheter'].mean()
test_dataset['failed_catheter'] = np.random.binomial(n=1, p=failure_rate, size=len(test_dataset))

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report

# Load the dataset
data = test_dataset

# Drop columns with more than 80% missing values
threshold = len(data) * 0.2
data_cleaned = data.dropna(thresh=threshold, axis=1)

# Drop rows where target variable is missing
data_cleaned = data_cleaned.dropna(subset=["failed_catheter"])

# Separate features and target variable
X = data_cleaned.drop(columns=["failed_catheter"], errors='ignore')
y = data_cleaned["failed_catheter"]

##############################################################################
# 1. Extract the group labels and remove them from X if it's just an ID column
##############################################################################
groups = X['unique_pt_id']  # Save group labels
# If you do NOT want to use `anes_procedure_encounter_id_2273` as a feature:
X = X.drop(columns=["unique_pt_id"])  

##############################################################################
# 2. Split using GroupShuffleSplit instead of train_test_split
##############################################################################
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Identify numeric and categorical columns
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'bool']).columns.tolist()

# Create preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing into a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Train logistic regression with class weights
logistic_model = LogisticRegression(max_iter=1000, solver='liblinear', class_weight='balanced', n_jobs=1)
logistic_model.fit(X_train_preprocessed, y_train)

# Make predictions
y_pred = logistic_model.predict(X_test_preprocessed)
y_pred_prob = logistic_model.predict_proba(X_test_preprocessed)[:, 1]

# Evaluate the model
evaluation_metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred),
    "recall": recall_score(y_test, y_pred),
    "roc_auc": roc_auc_score(y_test, y_pred_prob),
    "classification_report": classification_report(y_test, y_pred)
}

# Print evaluation metrics
print("TEST RANDOM Model Evaluation:")
for metric, value in evaluation_metrics.items():
    if metric == "classification_report":
        print("\nTEST RANDOM Classification Report:\n", value)
    else:
        print(f"{metric.capitalize()}: {value:.4f}")


### Real LOGIT regression

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GroupShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report

import statsmodels.api as sm

# 1. Load and clean data
data = neuraxial_catheter_df.copy()

# Drop columns with more than 80% missing values
threshold = len(data) * 0.2
data_cleaned = data.dropna(thresh=threshold, axis=1)
data_cleaned = data_cleaned.dropna(subset=["failed_catheter"])

X = data_cleaned.drop(columns=["failed_catheter"], errors='ignore')
y = data_cleaned["failed_catheter"]

# 2. Group-aware split
groups = X['unique_pt_id']
X = X.drop(columns=["anes_procedure_encounter_id_2273","unique_pt_id"])

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()

# 3. Identify numeric vs. categorical
numeric_features = X_train.select_dtypes(include=["float64", "int64"]).columns.tolist()
categorical_features = X_train.select_dtypes(include=["object", "bool"]).columns.tolist()

# 4. Impute numeric
num_imputer = SimpleImputer(strategy='median')
X_train[numeric_features] = num_imputer.fit_transform(X_train[numeric_features])
X_test[numeric_features] = num_imputer.transform(X_test[numeric_features])

# 5. Scale numeric
scaler = StandardScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

# 6. Impute categorical
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train[categorical_features] = cat_imputer.fit_transform(X_train[categorical_features])
X_test[categorical_features] = cat_imputer.transform(X_test[categorical_features])

# 7. One-hot encode categorical
X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True,dtype=int).astype(float)
X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True,dtype=int).astype(float)

X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

# 8. Fit logistic regression with Statsmodels
X_train_const = sm.add_constant(X_train)
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit(disp=0)

# 9. Predict
X_test_const = sm.add_constant(X_test, has_constant='add')
y_pred_prob = result.predict(X_test_const)

evaluation_metrics_by_threshold = []

for i in range(0, 21):
    prediction_threshold = i / 20
    y_pred = (y_pred_prob >= prediction_threshold).astype(int)

    # 10. Evaluate
    evaluation_metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred,zero_division=np.nan),
        "recall": recall_score(y_test, y_pred),
        "roc_auc": roc_auc_score(y_test, y_pred_prob),
        # "classification_report": classification_report(y_test, y_pred)
    }
    evaluation_metrics_by_threshold.append(evaluation_metrics)

result_summ = result.summary(alpha = 0.001)

# print("Model Evaluation:")
# for metric, value in evaluation_metrics.items():
#     if metric == "classification_report":
#         print("\nClassification Report:\n", value)
#     else:
#         print(f"{metric.capitalize()}: {value:.4f}")


In [ ]:
evaluation_metrics_by_threshold = pd.DataFrame(evaluation_metrics_by_threshold)
evaluation_metrics_by_threshold

In [ ]:
logit_result_table = result_summ.tables[0].data

In [ ]:
logit_predictor_table = pd.DataFrame(result_summ.tables[1].data)
# 1. Set the first row as the new column headers.
logit_predictor_table.columns = logit_predictor_table.iloc[0]

# 2. Remove the first row (since it's now serving as header).
logit_predictor_table = logit_predictor_table[1:]

# 3. Set the first column (after the column headers update) as the index.
# Here, `df.columns[0]` represents the name of the first column.
logit_predictor_table = logit_predictor_table.set_index(logit_predictor_table.columns[0])

# 4. Sort by the 'P>|z|' column in ascending order.
logit_predictor_table = logit_predictor_table.sort_values('P>|z|')


In [ ]:
# Calculate the odds ratio (OR) and the 95% confidence intervals
logit_predictor_table['OR'] = np.exp(logit_predictor_table['coef'].astype(float))
logit_predictor_table['OR_lower'] = np.exp(logit_predictor_table['[0.0005'].astype(float))
logit_predictor_table['OR_upper'] = np.exp(logit_predictor_table['0.9995]'].astype(float))

# Create the 'OR (95% CI)' column
logit_predictor_table['OR (99.9% CI)'] = logit_predictor_table.apply(
    lambda row: f"{row['OR']:.2f} ({row['OR_lower']:.2f} - {row['OR_upper']:.2f})", axis=1)

logit_predictor_table

In [ ]:
logit_predictor_table.shape

In [ ]:
logit_predictor_table.index

In [ ]:
patient_factors = ['delivery_site_bwh',
       'parity_2048', 'gestational_age_weeks',
       'presentation_cephalic',
       'has_dorsalgia',
       'maternal_age_years', 'has_back_problems',
       'prior_failed_catheters_prev_enc',
       'baby_weight_2196',
       'composite_psychosocial_problems', 'prior_failed_catheters_this_enc',
       'position_posterior_or_transverse', 'prior_pain_scores_max',
       'bmi_end_pregnancy_2044', 'labor_induction']

procedural_factors = ['true_procedure_type_incl_dpe_cse', 
       'true_procedure_type_incl_dpe_dpe',
       'highly_experienced_anesthesiologist_none', 'lor_depth', 'number_of_neuraxial_attempts',
       'paresthesias_present',
       'highly_experienced_resident_no',
       'highly_experienced_anesthesiologist_yes',
       'highly_experienced_resident_yes',
       'true_procedure_type_incl_dpe_intrathecal']

In [ ]:
patient_df = logit_predictor_table.loc[patient_factors].copy()
procedural_df = logit_predictor_table.loc[procedural_factors].copy()

rename_map = {
    'gestational_age_weeks': 'Gestational Age (per week)',
    'delivery_site_bwh': 'Delivery at our obstetric teaching hospital (vs other)',
    'baby_weight_2196': 'Baby Weight (per kg)',
    'bmi_end_pregnancy_2044': 'BMI (per kg/m^2)',
    'parity_2048': 'Parity (per birth)',
    'has_dorsalgia': 'Back pain (vs none)',
    'has_back_problems': 'Scoliosis or other back problems (vs none)',
    'prior_pain_scores_max': 'Max pain score prior to placement (per unit 0-10)',
    'composite_psychosocial_problems': 'Psychosocial risk factors (vs none)',
    'prior_failed_catheters_this_enc': 'Prior failed catheters in this encounter (per failure)',
    'prior_failed_catheters_prev_enc': 'Prior failed catheters in prior encounters (per failure)',
    'maternal_age_years': 'Maternal age (per year)',
    'labor_induction': 'Induced labor (vs not)',
    'position_posterior_or_transverse': 'Posterior or transverse fetal position (vs other)',
    'presentation_cephalic': 'Cephalic fetal presentation (vs other)',
    # procedural factors below
    'lor_depth': 'Depth to loss of resistance (per cm)',
    'highly_experienced_anesthesiologist_yes': 'Highly experienced attending anesthesiologist (vs less experienced)',
    'highly_experienced_anesthesiologist_none': 'No attending anesthesiologist (vs less experienced attending)',
    'highly_experienced_resident_yes': 'Highly experienced resident (vs no resident)',
    'highly_experienced_resident_no': 'Less experienced resident (vs no resident)',
    'paresthesias_present': 'Paresthesias present during placement (vs none)',
    'number_of_neuraxial_attempts': 'Number of placement attempts (per attempt)',
    'true_procedure_type_incl_dpe_intrathecal': 'Intrathecal catheter (vs conventional epidural)',
    'true_procedure_type_incl_dpe_dpe': 'Dural puncture epidural (vs conventional epidural)',
    'true_procedure_type_incl_dpe_cse': 'Combined spinal-epidural (vs conventional epidural)'
}

patient_df = patient_df.rename(index=rename_map)
procedural_df = procedural_df.rename(index=rename_map)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

def plot_forest_colored_with_markers(
    ax, 
    df, 
    title='Forest Plot', 
    x_label='Odds Ratio (99.9% CI)',
    x_min=0.5, 
    x_max=1.5
):
    """
    Plot a forest chart on 'ax' given a DataFrame 'df' with columns:
      - 'OR'
      - 'OR_lower'
      - 'OR_upper'
    
    X-axis is restricted to [x_min, x_max].
    
    Rules:
      - If the center OR is outside [x_min, x_max], skip plotting its dot.
      - If the OR or any part of its CI is beyond [x_min, x_max], place '<' or '>' at that boundary.
      - Print "OR X.XX (L.LL, U.UU)" above each data point in the same color.
      - Color each factor's name on the y-axis to match that factor's color.
    
    Color scheme for significance:
      - red if entire CI > 1
      - blue if entire CI < 1
      - black otherwise
    """

    # Sort by OR if you want smaller/larger ORs in order
    df = df.sort_values('OR')

    # We'll manually set the y-ticks, one row per factor
    y_positions = np.arange(len(df))

    # We won't set the yticklabels yet; we'll do them manually to color each label.
    ax.set_yticks(y_positions)
    # Temporarily set them all to blank
    ax.set_yticklabels([""] * len(df))

    # We'll collect the color for each row, so we can color the labels afterward
    factor_colors = []

    # Plot each factor individually
    for y_pos, (idx, row) in zip(y_positions, df.iterrows()):
        or_val = row['OR']
        ci_low = row['OR_lower']
        ci_high = row['OR_upper']

        # Decide the color based on significance
        if ci_low > 1:
            c = 'red'    # entire CI above 1 => significant risk
        elif ci_high < 1:
            c = 'blue'   # entire CI below 1 => significant protective
        else:
            c = 'black'  # not significant

        factor_colors.append(c)

        # Check if OR or CI extends beyond the plot range
        outside_left = (or_val < x_min) or (ci_low < x_min)
        outside_right = (or_val > x_max) or (ci_high > x_max)

        # If the center OR is out of range, skip the dot
        center_outside = (or_val < x_min) or (or_val > x_max)
        dot_fmt = 'none' if center_outside else 'o'

        # Calculate the full error bar from the center
        left_err = or_val - ci_low
        right_err = ci_high - or_val

        # Plot the error bar (may or may not include the dot)
        ax.errorbar(
            or_val,
            y_pos,
            xerr=[[left_err], [right_err]],
            fmt=dot_fmt,   # skip the dot if center is outside
            color=c,
            ecolor=c,
            capsize=4
        )

        # Place boundary markers if the OR or any part of CI is outside
        if outside_left:
            ax.text(
                x_min, y_pos, '<', 
                va='center', ha='right', color=c, fontsize=14
            )
        if outside_right:
            ax.text(
                x_max, y_pos, '>', 
                va='center', ha='left', color=c, fontsize=14
            )

        # Prepare the label "OR X.XX (L.LL - U.UU)"
        label_str = f"OR {or_val:.2f} ({ci_low:.2f} - {ci_high:.2f})"

        # Place the label just above the data point (or boundary)
        # We'll define a small offset in Y to shift text "above" the marker
        label_offset = 0.2  # Adjust as needed
        label_y = y_pos - label_offset  # axis is inverted => subtract to go "up"
        ax.text(
            1.06, label_y,
            label_str,
            va='bottom',   # text rises from the point
            ha='center',
            color=c,
            fontsize=10
        )

    # Now color each factor name using the same color
    # We already set blank y-ticklabels, so let's manually place them:
    for y_pos, (idx, c) in zip(y_positions, zip(df.index, factor_colors)):
        # We'll place the text a bit left of x_min so it doesn't collide with the plot
        ax.text(
            x_min - 0.05, y_pos,
            idx,
            va='center', ha='right',
            color=c,
            fontsize=10
        )

    # Draw a vertical line at OR=1
    ax.axvline(x=1, color='gray', linestyle='--')

    # Invert y-axis so the top row is at the top
    ax.invert_yaxis()

    # Limit the x-axis
    ax.set_xlim([x_min, x_max])

    # Add labels
    ax.set_xlabel(x_label, fontsize=14)
    ax.set_title(title, fontsize=16)


# ==========================
# Example usage with TWO subplots
# ==========================

# Suppose you have:
#   patient_df
#   procedural_df
# Each with columns: ['OR', 'OR_lower', 'OR_upper']
# and index = factor names.

# Create the figure with two columns
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 8))

plot_forest_colored_with_markers(
    ax=ax1,
    df=patient_df,
    title='Patient Factors',
    x_label='Odds Ratio (99.9% CI)',
    x_min=0.5,
    x_max=1.5
)

plot_forest_colored_with_markers(
    ax=ax2,
    df=procedural_df,
    title='Procedural Factors',
    x_label='Odds Ratio (99.9% CI)',
    x_min=0.5,
    x_max=1.5
)

# Create a manual legend for color interpretation:
protect_marker = mlines.Line2D([], [], color='blue', marker='o', linestyle='None',
                               label='Significant protective factor')
ns_marker = mlines.Line2D([], [], color='black', marker='o', linestyle='None',
                          label='Not significant')
risk_marker = mlines.Line2D([], [], color='red', marker='o', linestyle='None',
                            label='Significant risk factor')

fig.legend(
    handles=[protect_marker, ns_marker, risk_marker],
    loc='upper center',
    bbox_to_anchor=(0.5, 1.05),
    ncol=3,
    fontsize=12
)

plt.tight_layout()
plt.show()


## SKLearn Logistic Regression

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report

# Load the dataset
data = neuraxial_catheter_df

# Drop columns with more than 80% missing values
threshold = len(data) * 0.2
data_cleaned = data.dropna(thresh=threshold, axis=1)

# Drop rows where target variable is missing
data_cleaned = data_cleaned.dropna(subset=["failed_catheter"])

# Separate features and target variable
X = data_cleaned.drop(columns=["failed_catheter"], errors='ignore')
y = data_cleaned["failed_catheter"]


## Drop delivery_site
# X = X.drop(columns=["delivery_site"],errors='ignore')

##############################################################################
# 1. Extract the group labels and remove them from X if it's just an ID column
##############################################################################
groups = X['unique_pt_id']  # Save group labels
# If you do NOT want to use `anes_procedure_encounter_id_2273` as a feature:
X = X.drop(columns=["anes_procedure_encounter_id_2273","unique_pt_id"])  

##############################################################################
# 2. Split using GroupShuffleSplit instead of train_test_split
##############################################################################
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Identify numeric and categorical columns
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'bool']).columns.tolist()

# Create preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing into a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Train logistic regression with class weights
logistic_model = LogisticRegression(max_iter=1000, solver='liblinear', class_weight='balanced', n_jobs=1)
logistic_model.fit(X_train_preprocessed, y_train)

# Make predictions
y_pred = logistic_model.predict(X_test_preprocessed)
y_pred_prob = logistic_model.predict_proba(X_test_preprocessed)[:, 1]

# Evaluate the model
evaluation_metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred),
    "recall": recall_score(y_test, y_pred),
    "roc_auc": roc_auc_score(y_test, y_pred_prob),
    "classification_report": classification_report(y_test, y_pred)
}

# Print evaluation metrics
print("Model Evaluation:")
for metric, value in evaluation_metrics.items():
    if metric == "classification_report":
        print("\nClassification Report:\n", value)
    else:
        print(f"{metric.capitalize()}: {value:.4f}")


In [ ]:
# 1. Get the feature names produced by the ColumnTransformer
feature_names = preprocessor.get_feature_names_out()

# 2. Get the coefficients from the trained logistic model
#    For binary classification, .coef_ is an array of shape (1, n_features)
coefficients = logistic_model.coef_[0]


# 3. Combine feature names with coefficients into a list of tuples
coef_pairs = list(zip(feature_names, coefficients))

# 4. Print the coefficients sorted by absolute magnitude
print("Coefficients for each feature (sorted by absolute magnitude):")
for name, coef in sorted(coef_pairs, key=lambda x: abs(x[1]), reverse=True):
    print(f"  {name}: {coef:.4f}")

# 5. Print the coefficients sorted alphabetically
print('---------------------------------------------')
print("Coefficients for each feature (sorted alphabetically):")
for name, coef in sorted(coef_pairs):
    print(f"  {name}: {coef:.4f}")

# 6. Print the intercept
print(f"Intercept: {logistic_model.intercept_[0]:.4f}")


# XGBoost

In [ ]:
##############################################################################
# Imports
##############################################################################
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report

# Import XGBoost
from xgboost import XGBClassifier

##############################################################################
# 0. Load and prepare the dataset
##############################################################################
data = neuraxial_catheter_df

# Drop columns with more than 80% missing values
threshold = len(data) * 0.2
data_cleaned = data.dropna(thresh=threshold, axis=1)

# Drop rows where target variable is missing
data_cleaned = data_cleaned.dropna(subset=["failed_catheter"])

# Separate features and target variable
X = data_cleaned.drop(columns=["failed_catheter"], errors='ignore')
y = data_cleaned["failed_catheter"]

# # Drop delivery_site
# X = X.drop(columns=["delivery_site"],errors='ignore')

##############################################################################
# 1. Extract the group labels and remove them from X if it's just an ID column
##############################################################################
groups = X['unique_pt_id']  # Save group labels
X = X.drop(columns=["anes_procedure_encounter_id_2273","unique_pt_id"])  # remove ID column from features

##############################################################################
# 2. Split using GroupShuffleSplit instead of train_test_split
##############################################################################
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

##############################################################################
# 3. Identify numeric and categorical columns
##############################################################################
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'bool']).columns.tolist()

##############################################################################
# 4. Create preprocessing pipelines
##############################################################################
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

##############################################################################
# 5. Preprocess the data
##############################################################################
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

##############################################################################
# 6. Train XGBoost Classifier
##############################################################################
# Instantiate the XGBClassifier
# Note: You can tune parameters such as 'scale_pos_weight' if your data is imbalanced.
xgb_model = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=(y_train.shape[0] - y_train.sum()) / y_train.sum()
)
xgb_model.fit(X_train_preprocessed, y_train)

##############################################################################
# 7. Make predictions
##############################################################################
y_pred = xgb_model.predict(X_test_preprocessed)
y_pred_prob = xgb_model.predict_proba(X_test_preprocessed)[:, 1]

##############################################################################
# 8. Evaluate the model
##############################################################################
evaluation_metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred),
    "recall": recall_score(y_test, y_pred),
    "roc_auc": roc_auc_score(y_test, y_pred_prob),
    "classification_report": classification_report(y_test, y_pred)
}

##############################################################################
# 9. Print the evaluation metrics
##############################################################################
print("Model Evaluation:")
for metric, value in evaluation_metrics.items():
    if metric == "classification_report":
        print("\nClassification Report:\n", value)
    else:
        print(f"{metric.capitalize()}: {value:.4f}")


# Propensity Scores

## Propensity Scoring for DPE

In [ ]:
import pandas as pd
import numpy as np

# For logistic regression and nearest neighbor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# For imputation and scaling
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# For statistical inference (CIs, p-values)
import statsmodels.api as sm

# ------------------------------------------------------------------------------
# 1. Copy your original dataframe
# ------------------------------------------------------------------------------
df = neuraxial_catheter_df.copy()
df['dpe'] = (df['true_procedure_type_incl_dpe'] == 'dpe').astype(int)
df.drop(columns=['true_procedure_type_incl_dpe'], inplace=True)

# Columns for the treatment and outcome
treatment_col = 'dpe'
outcome_col   = 'failed_catheter'

# ------------------------------------------------------------------------------
# 2. Identify numeric vs. categorical columns (excluding treatment & outcome)
# ------------------------------------------------------------------------------
# If 'dpe' or 'failed_catheter' happen to be numeric, we still exclude them from imputation.
numeric_cols = [
    col for col in df.select_dtypes(include=[np.number]).columns
    if col not in [treatment_col, outcome_col]
]
categorical_cols = [
    col for col in df.columns
    if col not in numeric_cols and col not in [treatment_col, outcome_col]
]

# ------------------------------------------------------------------------------
# 3. Impute missing data
#    - Median for numeric
#    - Most frequent for categorical
# ------------------------------------------------------------------------------
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Fit/transform numeric columns
df_num = pd.DataFrame(
    num_imputer.fit_transform(df[numeric_cols]),
    columns=numeric_cols
)

# Fit/transform categorical columns
df_cat = pd.DataFrame(
    cat_imputer.fit_transform(df[categorical_cols]),
    columns=categorical_cols
)

# ------------------------------------------------------------------------------
# 4. One-hot encode (dummy) the categorical columns
# ------------------------------------------------------------------------------
df_cat_encoded = pd.get_dummies(df_cat, drop_first=True)

# ------------------------------------------------------------------------------
# 5. Combine imputed numeric + encoded categorical with original treatment/outcome
# ------------------------------------------------------------------------------
# Reattach treatment/outcome columns to the front, for convenience
df_imputed = pd.concat(
    [
        df[[treatment_col, outcome_col]].reset_index(drop=True),
        df_num.reset_index(drop=True),
        df_cat_encoded.reset_index(drop=True)
    ],
    axis=1
)

# ------------------------------------------------------------------------------
# 6. Standardize numeric features (optional but often recommended)
#    Identify which columns in df_num still exist in df_imputed
# ------------------------------------------------------------------------------
scaler = StandardScaler()
df_num_scaled = pd.DataFrame(
    scaler.fit_transform(df_imputed[numeric_cols]),
    columns=numeric_cols
)

# Now replace the unscaled numeric columns in df_imputed
for col in numeric_cols:
    df_imputed[col] = df_num_scaled[col]

# ------------------------------------------------------------------------------
# 7. Fit the propensity model (LogisticRegression) on all columns except
#    the treatment and outcome columns.
# ------------------------------------------------------------------------------
feature_cols = [c for c in df_imputed.columns if c not in [treatment_col, outcome_col]]

X = df_imputed[feature_cols].values  # all imputed & encoded features
y = df_imputed[treatment_col].values # the treatment indicator (dpe)

propensity_model = LogisticRegression(solver='lbfgs', max_iter=1000)
propensity_model.fit(X, y)

# Probability of dpe=1
propensity_scores = propensity_model.predict_proba(X)[:, 1]
df_imputed['propensity_score'] = propensity_scores

# ------------------------------------------------------------------------------
# 8. Separate treated vs. control and do nearest-neighbor matching
# ------------------------------------------------------------------------------
treated = df_imputed[df_imputed[treatment_col] == 1].copy()
control = df_imputed[df_imputed[treatment_col] == 0].copy()

treated_scores = treated[['propensity_score']].values
control_scores = control[['propensity_score']].values

nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nn.fit(control_scores)

distances, indices = nn.kneighbors(treated_scores)
distances = distances.flatten()
indices = indices.flatten()

matched_treated = treated.copy()
matched_control = control.iloc[indices].copy()

# Combine matched sample
matched_data = pd.concat([matched_treated, matched_control], axis=0).reset_index(drop=True)

# ------------------------------------------------------------------------------
# 9. Fit an outcome model on the matched sample
#    We'll use statsmodels for confidence intervals and p-values.
# ------------------------------------------------------------------------------
matched_data['intercept'] = 1.0

# We'll just use dpe (and intercept) in the outcome model here
X_outcome = matched_data[['intercept', treatment_col]]
y_outcome = matched_data[outcome_col]

logit_sm = sm.Logit(y_outcome, X_outcome)
result_sm = logit_sm.fit(disp=0)  # disp=0 hides optimization output

print(result_sm.summary())

# Extract OR & 95% CI
params = result_sm.params
conf = result_sm.conf_int()
odds_ratios = np.exp(params)
conf_odds = np.exp(conf)

print("\nOdds Ratios:\n", odds_ratios)
print("\n95% Confidence Intervals:\n", conf_odds)


## Propensity Scoring for CSE

In [ ]:
import pandas as pd
import numpy as np

# For logistic regression and nearest neighbor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# For imputation and scaling
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# For statistical inference (CIs, p-values)
import statsmodels.api as sm

# ------------------------------------------------------------------------------
# 1. Copy your original dataframe
# ------------------------------------------------------------------------------
df = neuraxial_catheter_df.copy()
df['cse'] = (df['true_procedure_type_incl_dpe'] == 'cse').astype(int)
df.drop(columns=['true_procedure_type_incl_dpe'], inplace=True)

# Columns for the treatment and outcome
treatment_col = 'cse'
outcome_col   = 'failed_catheter'

# ------------------------------------------------------------------------------
# 2. Identify numeric vs. categorical columns (excluding treatment & outcome)
# ------------------------------------------------------------------------------
# If 'dpe' or 'failed_catheter' happen to be numeric, we still exclude them from imputation.
numeric_cols = [
    col for col in df.select_dtypes(include=[np.number]).columns
    if col not in [treatment_col, outcome_col]
]
categorical_cols = [
    col for col in df.columns
    if col not in numeric_cols and col not in [treatment_col, outcome_col]
]

# ------------------------------------------------------------------------------
# 3. Impute missing data
#    - Median for numeric
#    - Most frequent for categorical
# ------------------------------------------------------------------------------
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Fit/transform numeric columns
df_num = pd.DataFrame(
    num_imputer.fit_transform(df[numeric_cols]),
    columns=numeric_cols
)

# Fit/transform categorical columns
df_cat = pd.DataFrame(
    cat_imputer.fit_transform(df[categorical_cols]),
    columns=categorical_cols
)

# ------------------------------------------------------------------------------
# 4. One-hot encode (dummy) the categorical columns
# ------------------------------------------------------------------------------
df_cat_encoded = pd.get_dummies(df_cat, drop_first=True)

# ------------------------------------------------------------------------------
# 5. Combine imputed numeric + encoded categorical with original treatment/outcome
# ------------------------------------------------------------------------------
# Reattach treatment/outcome columns to the front, for convenience
df_imputed = pd.concat(
    [
        df[[treatment_col, outcome_col]].reset_index(drop=True),
        df_num.reset_index(drop=True),
        df_cat_encoded.reset_index(drop=True)
    ],
    axis=1
)

# ------------------------------------------------------------------------------
# 6. Standardize numeric features (optional but often recommended)
#    Identify which columns in df_num still exist in df_imputed
# ------------------------------------------------------------------------------
scaler = StandardScaler()
df_num_scaled = pd.DataFrame(
    scaler.fit_transform(df_imputed[numeric_cols]),
    columns=numeric_cols
)

# Now replace the unscaled numeric columns in df_imputed
for col in numeric_cols:
    df_imputed[col] = df_num_scaled[col]

# ------------------------------------------------------------------------------
# 7. Fit the propensity model (LogisticRegression) on all columns except
#    the treatment and outcome columns.
# ------------------------------------------------------------------------------
feature_cols = [c for c in df_imputed.columns if c not in [treatment_col, outcome_col]]

X = df_imputed[feature_cols].values  # all imputed & encoded features
y = df_imputed[treatment_col].values # the treatment indicator (dpe)

propensity_model = LogisticRegression(solver='lbfgs', max_iter=1000)
propensity_model.fit(X, y)

# Probability of dpe=1
propensity_scores = propensity_model.predict_proba(X)[:, 1]
df_imputed['propensity_score'] = propensity_scores

# ------------------------------------------------------------------------------
# 8. Separate treated vs. control and do nearest-neighbor matching
# ------------------------------------------------------------------------------
treated = df_imputed[df_imputed[treatment_col] == 1].copy()
control = df_imputed[df_imputed[treatment_col] == 0].copy()

treated_scores = treated[['propensity_score']].values
control_scores = control[['propensity_score']].values

nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nn.fit(control_scores)

distances, indices = nn.kneighbors(treated_scores)
distances = distances.flatten()
indices = indices.flatten()

matched_treated = treated.copy()
matched_control = control.iloc[indices].copy()

# Combine matched sample
matched_data = pd.concat([matched_treated, matched_control], axis=0).reset_index(drop=True)

# ------------------------------------------------------------------------------
# 9. Fit an outcome model on the matched sample
#    We'll use statsmodels for confidence intervals and p-values.
# ------------------------------------------------------------------------------
matched_data['intercept'] = 1.0

# We'll just use dpe (and intercept) in the outcome model here
X_outcome = matched_data[['intercept', treatment_col]]
y_outcome = matched_data[outcome_col]

logit_sm = sm.Logit(y_outcome, X_outcome)
result_sm = logit_sm.fit(disp=0)  # disp=0 hides optimization output

print(result_sm.summary())

# Extract OR & 95% CI
params = result_sm.params
conf = result_sm.conf_int()
odds_ratios = np.exp(params)
conf_odds = np.exp(conf)

print("\nOdds Ratios:\n", odds_ratios)
print("\n95% Confidence Intervals:\n", conf_odds)

del df

## Propensity scoring for DPE vs CSE

In [ ]:
import pandas as pd
import numpy as np

# For logistic regression and nearest neighbor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# For imputation and scaling
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# For statistical inference (CIs, p-values)
import statsmodels.api as sm

# ------------------------------------------------------------------------------
# 1. Copy your original dataframe
# ------------------------------------------------------------------------------
df = neuraxial_catheter_df.copy()
df = df[df['true_procedure_type_incl_dpe'].isin(['cse', 'dpe'])]
df['cse_not_dpe'] = (df['true_procedure_type_incl_dpe'] == 'cse').astype(int)
df.drop(columns=['true_procedure_type_incl_dpe'], inplace=True)

# Columns for the treatment and outcome
treatment_col = 'cse_not_dpe'
outcome_col   = 'failed_catheter'

# ------------------------------------------------------------------------------
# 2. Identify numeric vs. categorical columns (excluding treatment & outcome)
# ------------------------------------------------------------------------------
# If 'dpe' or 'failed_catheter' happen to be numeric, we still exclude them from imputation.
numeric_cols = [
    col for col in df.select_dtypes(include=[np.number]).columns
    if col not in [treatment_col, outcome_col]
]
categorical_cols = [
    col for col in df.columns
    if col not in numeric_cols and col not in [treatment_col, outcome_col]
]

# ------------------------------------------------------------------------------
# 3. Impute missing data
#    - Median for numeric
#    - Most frequent for categorical
# ------------------------------------------------------------------------------
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Fit/transform numeric columns
df_num = pd.DataFrame(
    num_imputer.fit_transform(df[numeric_cols]),
    columns=numeric_cols
)

# Fit/transform categorical columns
df_cat = pd.DataFrame(
    cat_imputer.fit_transform(df[categorical_cols]),
    columns=categorical_cols
)

# ------------------------------------------------------------------------------
# 4. One-hot encode (dummy) the categorical columns
# ------------------------------------------------------------------------------
df_cat_encoded = pd.get_dummies(df_cat, drop_first=True)

# ------------------------------------------------------------------------------
# 5. Combine imputed numeric + encoded categorical with original treatment/outcome
# ------------------------------------------------------------------------------
# Reattach treatment/outcome columns to the front, for convenience
df_imputed = pd.concat(
    [
        df[[treatment_col, outcome_col]].reset_index(drop=True),
        df_num.reset_index(drop=True),
        df_cat_encoded.reset_index(drop=True)
    ],
    axis=1
)

# ------------------------------------------------------------------------------
# 6. Standardize numeric features (optional but often recommended)
#    Identify which columns in df_num still exist in df_imputed
# ------------------------------------------------------------------------------
scaler = StandardScaler()
df_num_scaled = pd.DataFrame(
    scaler.fit_transform(df_imputed[numeric_cols]),
    columns=numeric_cols
)

# Now replace the unscaled numeric columns in df_imputed
for col in numeric_cols:
    df_imputed[col] = df_num_scaled[col]

# ------------------------------------------------------------------------------
# 7. Fit the propensity model (LogisticRegression) on all columns except
#    the treatment and outcome columns.
# ------------------------------------------------------------------------------
feature_cols = [c for c in df_imputed.columns if c not in [treatment_col, outcome_col]]

X = df_imputed[feature_cols].values  # all imputed & encoded features
y = df_imputed[treatment_col].values # the treatment indicator (dpe)

propensity_model = LogisticRegression(solver='lbfgs', max_iter=1000)
propensity_model.fit(X, y)

# Probability of dpe=1
propensity_scores = propensity_model.predict_proba(X)[:, 1]
df_imputed['propensity_score'] = propensity_scores

# ------------------------------------------------------------------------------
# 8. Separate treated vs. control and do nearest-neighbor matching
# ------------------------------------------------------------------------------
treated = df_imputed[df_imputed[treatment_col] == 1].copy()
control = df_imputed[df_imputed[treatment_col] == 0].copy()

treated_scores = treated[['propensity_score']].values
control_scores = control[['propensity_score']].values

nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nn.fit(control_scores)

distances, indices = nn.kneighbors(treated_scores)
distances = distances.flatten()
indices = indices.flatten()

matched_treated = treated.copy()
matched_control = control.iloc[indices].copy()

# Combine matched sample
matched_data = pd.concat([matched_treated, matched_control], axis=0).reset_index(drop=True)

# ------------------------------------------------------------------------------
# 9. Fit an outcome model on the matched sample
#    We'll use statsmodels for confidence intervals and p-values.
# ------------------------------------------------------------------------------
matched_data['intercept'] = 1.0

# We'll just use dpe (and intercept) in the outcome model here
X_outcome = matched_data[['intercept', treatment_col]]
y_outcome = matched_data[outcome_col]

logit_sm = sm.Logit(y_outcome, X_outcome)
result_sm = logit_sm.fit(disp=0)  # disp=0 hides optimization output

print(result_sm.summary())

# Extract OR & 95% CI
params = result_sm.params
conf = result_sm.conf_int()
odds_ratios = np.exp(params)
conf_odds = np.exp(conf)

print("\nOdds Ratios:\n", odds_ratios)
print("\n95% Confidence Intervals:\n", conf_odds)

del df

# Next Steps

XGBoost - hyperparameter tuning with Optuna

Shapley values for interpretability

Eliminate features that are both poorly predictive and have lots of missing data

Abstract functions and separate them into different files

Fewer features will improve interpretability